In [1]:
import cv2
import numpy as np

In [2]:
def nothing(x):
    # any operation
    pass

In [3]:
def get_limits(color, l_limit, u_limit):
    c = np.uint8([[color]]) # here insert the bgr values which you want to convert to hsv
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    lowerLimit = hsvC[0][0][0] - l_limit, 40, 0
    upperLimit = hsvC[0][0][0] + u_limit, 255, 255

    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [25]:
def rescale_frame(frame_input, percent=100):
    width = int(frame_input.shape[1] * percent / 100)
    height = int(frame_input.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame_input, dim, interpolation=cv2.INTER_AREA)

In [26]:
cap = cv2.VideoCapture(1)


cv2.namedWindow("Trackbars")
cv2.createTrackbar("L-L-yellow", "Trackbars", 10, 255, nothing)
cv2.createTrackbar("U-L-yellow", "Trackbars", 10, 255, nothing)
cv2.createTrackbar("L-L-red", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U-L-red", "Trackbars", 5, 255, nothing)
cv2.createTrackbar("L-L-green", "Trackbars", 20, 255, nothing)
cv2.createTrackbar("U-L-green", "Trackbars", 20, 255, nothing)
cv2.createTrackbar("L-L-blue", "Trackbars", 25, 255, nothing)
cv2.createTrackbar("U-L-blue", "Trackbars", 25, 255, nothing)

# cap = cv2.VideoCapture("IMG_1178.MOV")
# cv2.createTrackbar("L-L-yellow", "Trackbars", 10, 255, nothing)
# cv2.createTrackbar("U-L-yellow", "Trackbars", 10, 255, nothing)
# cv2.createTrackbar("L-L-red", "Trackbars", 0, 255, nothing)
# cv2.createTrackbar("U-L-red", "Trackbars", 0, 255, nothing)
# cv2.createTrackbar("L-L-green", "Trackbars", 0, 255, nothing)
# cv2.createTrackbar("U-L-green", "Trackbars", 0, 255, nothing)
# cv2.createTrackbar("L-L-blue", "Trackbars", 25, 255, nothing)
# cv2.createTrackbar("U-L-blue", "Trackbars", 25, 255, nothing)

# in BGR colorspace
yellow = [0, 255, 255]
red = [0, 0, 255]
green = [0, 255, 0]
blue = [255, 0, 0]

while True:
    ret, frame = cap.read()
    if frame is None:
        break

    frame = rescale_frame(frame)

    hsvImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    l_limit_yellow = cv2.getTrackbarPos("L-L-yellow", "Trackbars")
    u_limit_yellow = cv2.getTrackbarPos("U-L-yellow", "Trackbars")
    l_limit_red = cv2.getTrackbarPos("L-L-red", "Trackbars")
    u_limit_red = cv2.getTrackbarPos("U-L-red", "Trackbars")
    l_limit_green = cv2.getTrackbarPos("L-L-green", "Trackbars")
    u_limit_green = cv2.getTrackbarPos("U-L-green", "Trackbars")
    l_limit_blue = cv2.getTrackbarPos("L-L-blue", "Trackbars")
    u_limit_blue = cv2.getTrackbarPos("U-L-blue", "Trackbars")

    lowerLimit_yellow, upperLimit_yellow = get_limits(yellow, l_limit_yellow, u_limit_yellow)
    lowerLimit_red, upperLimit_red = get_limits(red, l_limit_red, u_limit_red)
    lowerLimit_green, upperLimit_green = get_limits(green, u_limit_green, u_limit_green)
    lowerLimit_blue, upperLimit_blue = get_limits(blue, l_limit_blue, u_limit_blue)

    mask_yellow = cv2.inRange(hsvImage, lowerLimit_yellow, upperLimit_yellow)
    mask_red = cv2.inRange(hsvImage, lowerLimit_red, upperLimit_red)
    mask_green = cv2.inRange(hsvImage, lowerLimit_green, upperLimit_green)
    mask_blue = cv2.inRange(hsvImage, lowerLimit_blue, upperLimit_blue)

    # add masks
    # result = mask_yellow + mask_red + mask_green + mask_blue
    result = cv2.bitwise_or(mask_yellow,mask_red)
    result = cv2.bitwise_or(result,mask_green)
    result = cv2.bitwise_or(result,mask_blue)

    cv2.imshow("Frame", frame)
    # cv2.imshow("mask_yellow", mask_yellow)
    # cv2.imshow("mask_red", mask_red)
    # cv2.imshow("mask_green", mask_green)
    # cv2.imshow("mask_blue", mask_blue)
    cv2.imshow("result", result)
    if cv2.waitKey(30) == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()